In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Dense, Activation

In [47]:
X = np.random.random((10000, 2))

In [48]:
X.shape

(10000, 2)

In [49]:
M = np.dot(X, [[2, 3, 1], 
               [4, 2, 1]]) + 1

In [50]:
M.shape

(10000, 3)

In [51]:
Y = np.dot(M, [[1], 
               [5], 
               [2]]) + 1

In [52]:
Y.shape

(10000, 1)

In [53]:
n = X.shape[0]

In [54]:
rnd = np.random.random(n)
idx_trn = rnd < 0.8
idx_val = rnd >=0.8

In [55]:
X_trn = X[idx_trn]
X_val = X[idx_val]
Y_trn = Y[idx_trn]
Y_val = Y[idx_val]
M_trn = M[idx_trn]
M_val = M[idx_val]

## base model

In [56]:
base_model = Sequential([
    Dense(3, input_shape=(2,))
])

In [57]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [58]:
base_model.compile(optimizer='sgd', loss='mse')
base_model.fit(X_trn, M_trn, epochs=5, verbose=1, validation_data=(X_val, M_val))

Train on 8052 samples, validate on 1948 samples
Epoch 1/5
8052/8052 [==============================] - 0s - loss: 2.7576 - val_loss: 0.4104
Epoch 2/5
8052/8052 [==============================] - 0s - loss: 0.3115 - val_loss: 0.2745
Epoch 3/5
8052/8052 [==============================] - 0s - loss: 0.2419 - val_loss: 0.2222
Epoch 4/5
8052/8052 [==============================] - 0s - loss: 0.1963 - val_loss: 0.1807
Epoch 5/5
8052/8052 [==============================] - 0s - loss: 0.1597 - val_loss: 0.1472


freeze base_model weights

In [59]:
for layer in base_model.layers:
    layer.trainable = False

In [60]:
for layer in base_model.layers:
    print(layer.get_weights())

[array([[ 1.71209252,  1.92582953,  0.93598926],
       [ 2.20336413,  1.23195589,  0.86359763]], dtype=float32), array([ 2.10873652,  1.97526693,  1.1063906 ], dtype=float32)]


In [61]:
pred_mid_trn = base_model.predict(X_trn)
pred_mid_val = base_model.predict(X_val)

## base_model output as dense layer input

In [65]:
lm = Sequential([
    Dense(1, input_shape=(3,))
])
lm.compile(optimizer='sgd', loss='mse')

In [66]:
lm.fit(pred_mid_trn, Y_trn, epochs=10, verbose=1, validation_data=(pred_mid_val, Y_val))

Train on 8052 samples, validate on 1948 samples
Epoch 1/10
8052/8052 [==============================] - 0s - loss: 8.0680 - val_loss: 3.9051
Epoch 2/10
8052/8052 [==============================] - 0s - loss: 3.3304 - val_loss: 2.8588
Epoch 3/10
8052/8052 [==============================] - 0s - loss: 2.3988 - val_loss: 2.0220
Epoch 4/10
8052/8052 [==============================] - 0s - loss: 1.7408 - val_loss: 1.5464
Epoch 5/10
8052/8052 [==============================] - 0s - loss: 1.2600 - val_loss: 1.1351
Epoch 6/10
8052/8052 [==============================] - 0s - loss: 0.9303 - val_loss: 0.7881
Epoch 7/10
8052/8052 [==============================] - 0s - loss: 0.6843 - val_loss: 0.6065
Epoch 8/10
8052/8052 [==============================] - 0s - loss: 0.5043 - val_loss: 0.4251
Epoch 9/10
8052/8052 [==============================] - 0s - loss: 0.3746 - val_loss: 0.3155
Epoch 10/10
8052/8052 [==============================] - 0s - loss: 0.2786 - val_loss: 0.2495


In [67]:
for layer in lm.layers:
    print(layer.get_weights())

[array([[ 3.46466136],
       [ 4.88001966],
       [ 2.15676951]], dtype=float32), array([-9.29623032], dtype=float32)]


## put in one model 

In [70]:
x = base_model.output
predictions = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='sgd', loss='mse')

In [71]:
model.fit(X_trn, Y_trn, epochs=10, verbose=1, validation_data=(X_val, Y_val))

Train on 8052 samples, validate on 1948 samples
Epoch 1/10
8052/8052 [==============================] - 0s - loss: 7.8509 - val_loss: 3.8569
Epoch 2/10
8052/8052 [==============================] - 0s - loss: 3.2547 - val_loss: 2.7461
Epoch 3/10
8052/8052 [==============================] - 0s - loss: 2.3458 - val_loss: 1.9946
Epoch 4/10
8052/8052 [==============================] - 0s - loss: 1.6857 - val_loss: 1.4470
Epoch 5/10
8052/8052 [==============================] - 0s - loss: 1.2341 - val_loss: 1.0412
Epoch 6/10
8052/8052 [==============================] - 0s - loss: 0.8983 - val_loss: 0.7872
Epoch 7/10
8052/8052 [==============================] - 0s - loss: 0.6590 - val_loss: 0.5669
Epoch 8/10
8052/8052 [==============================] - 0s - loss: 0.4862 - val_loss: 0.4272
Epoch 9/10
8052/8052 [==============================] - 0s - loss: 0.3594 - val_loss: 0.3033
Epoch 10/10
8052/8052 [==============================] - 0s - loss: 0.2677 - val_loss: 0.2294


In [72]:
for layer in model.layers:
    print(layer.get_weights())

[]
[array([[ 1.71209252,  1.92582953,  0.93598926],
       [ 2.20336413,  1.23195589,  0.86359763]], dtype=float32), array([ 2.10873652,  1.97526693,  1.1063906 ], dtype=float32)]
[array([[ 3.80313993],
       [ 5.39420986],
       [ 0.55602229]], dtype=float32), array([-9.23373795], dtype=float32)]


In [73]:
base_model.layers[0].get_weights()

[array([[ 1.71209252,  1.92582953,  0.93598926],
        [ 2.20336413,  1.23195589,  0.86359763]], dtype=float32),
 array([ 2.10873652,  1.97526693,  1.1063906 ], dtype=float32)]

In [74]:
np.all(model.layers[1].get_weights()[0] == base_model.layers[0].get_weights()[0])

True

In [75]:
np.all(model.layers[1].get_weights()[1] == base_model.layers[0].get_weights()[1])

True

In [76]:
model.layers[2].get_weights()

[array([[ 3.80313993],
        [ 5.39420986],
        [ 0.55602229]], dtype=float32), array([-9.23373795], dtype=float32)]

In [77]:
lm.layers[0].get_weights()

[array([[ 3.46466136],
        [ 4.88001966],
        [ 2.15676951]], dtype=float32), array([-9.29623032], dtype=float32)]

### output of model at each layer from X_trn

In [103]:
# https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp]+ [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

# Testing
layer_outs = [func([X_trn, 1.]) for func in functors]

In [104]:
np.allclose(X_trn, layer_outs[0])

True

In [105]:
np.all(pred_mid_trn == layer_outs[1])

True

In [106]:
pred_trn = model.predict(X_trn)

In [107]:
pred_trn

array([[ 16.15377617],
       [ 14.06655502],
       [ 36.16280365],
       ..., 
       [ 18.99808121],
       [ 27.932827  ],
       [ 25.92883301]], dtype=float32)

In [108]:
np.all(pred_trn == layer_outs[2])

True

### output of model at each layer from X_val

In [88]:
# https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp]+ [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

# Testing
layer_outs = [func([X_val, 1.]) for func in functors]

In [89]:
np.allclose(X_val, layer_outs[0])

True

In [90]:
np.all(pred_mid_val == layer_outs[1])

True

In [98]:
pred_val = model.predict(X_val)

In [99]:
np.all(pred_val == layer_outs[2])

True